[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/repos-especializacion-UdeA/data-raw/blob/main/notebooks/merge_databases.ipynb)

# Combinación de todas las bases de datos

El siguiente notebook explora de manera sencilla un archivo de matlab donde se guarda la información de un sensor.

In [7]:
try:
    import scipy.io
except ImportError:
    !pip install scipy

try:
    from ydata_profiling import ProfileReport
except ImportError:
    !pip install ydata_profiling

## 1. Librerias y configuraciones previas

In [8]:
import sys
import os
import zipfile
from ydata_profiling import ProfileReport

# Get the absolute path of the current notebook
notebook_path = "."
print(notebook_path)
try:
    import google.colab
    if not(os.path.exists("/content/data-raw/notebooks")):
        !git clone https://github.com/repos-especializacion-UdeA/data-raw.git
    %cd /content/data-raw/notebooks   
    %pwd
    ruta_base = '/content/data-raw/notebooks/'
    sys.path.append(ruta_base)
except ImportError:
    print("El notebook no se está ejecutando en Google Colab.")
    ruta_base = './'

ImportError: C extension: None not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [9]:
# command to view figures in Jupyter notebook
# %matplotlib inline 

# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np
import scipy as sc

# Almacenar en caché los resultados de funciones en el disco
# ==============================================================================
import joblib


# Gestion de librerias
# ==============================================================================
from importlib import reload

# Matemáticas y estadísticas
# ==============================================================================
import math

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns


# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

# Formateo y estilo
# ==============================================================================
from IPython.display import Markdown, display

# Biblioteca scipy y componentes
# ==============================================================================
import scipy.io
from scipy import signal


ImportError: C extension: None not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

## 2. Funciones

In [ ]:
# Funciones externas
# ==============================================================================
# from utils.funciones1 import multiple_plot
# from utils.funciones2 import test_hola, graficar_medida, graficar_medida2, filter_signal, \
#                             segmentar_data_base, aplanar_data_base, aplanar_ventana

In [ ]:
# test_hola()

## 3. Carga del dataset

Inicialmente se verifica que el archivo csv este disponible.

In [ ]:
# Verificando que se encuentre el archivo

DATASETS_PATH = "./datasets/"
dataset_zip = "raw_dataset.zip"
dataset_csv = "raw_dataset.csv"

if not(os.path.exists(DATASETS_PATH + dataset_zip)):
    # Descoprimir dataset
    with zipfile.ZipFile(DATASETS_PATH + dataset_zip, 'r') as zip_ref:
        zip_ref.extractall(".")
else:
    print("El dataset existe")
    

A continuación se realiza la carga del dataset completo

In [ ]:
# Carga del dataset
df = pd.read_csv(DATASETS_PATH + dataset_csv)

A continuación se verifica la carga del dataset:

In [ ]:
# Mostrar las primeras filas del DataFrame
df.head()

In [ ]:
df.info()

Hay un total de 13 columnas y ninguna tiene registros faltantes (missing values). Debido a esto, no nos tendremos que preocupar por realizar una imputación de datos.

Es importante aclarar que las columnas `s` (sujeto), `rep` (repeticion) y `label` (postura) estan relacionadas con datos categoricos por lo que vamos a proceder a realizar la transformación de estos.

In [ ]:
# Convertir a categorico
df['s'] = pd.Categorical(df['s'])
df['rep'] = pd.Categorical(df['rep'])
df['label'] = pd.Categorical(df['label'])

Se verifica que los cambios en el dataframe se hayan efectuado.

In [ ]:
#Lista de variables categóricas
catCols = df.select_dtypes(include = ['object', 'category']).columns.tolist()
print(f"Variables categoricas: {catCols}")
numCols = df.select_dtypes(include = ['float64','int32','int64']).columns.tolist()
print(f"Variables categoricas: {numCols}")

## Contextualización de los datos

Antes de seguir, como los datos a trabajar son señales se va a proceder a realizar unas graficas temporales de la señal para poner en contexto al lector. 

Es importante tener en cuenta que la frecuencia de muestro de la señal es de 100 Hz

In [ ]:
sample_freq = 100

Vamos a agregar una columna asociada al tiempo en segundos en la que se hace la muestra:

In [ ]:
# Columna del tiempo
df['t'] = (1/sample_freq)*df.index

In [ ]:
df.head()

Tomemos las muestras asociadas al sujeto 1:

In [ ]:
df_s1 = df[df['s'] == 1]

Graficando la señales asociadas a los sensores sEMG

In [ ]:
emg_signals_s1 = df_s1.iloc[:,1:11].values
timeEMG_s1 = df_s1['t']
repeticiones_s1 = df_s1['rep']
posturas_s1 = df_s1['label']

In [ ]:
# Crear la figura y subgráficos
fig, ax = plt.subplots(figsize=(15, 5))
# print(emg_signals_s1)

for i in range(emg_signals_s1.shape[1]):
    ax.plot(timeEMG_s1, emg_signals_s1[:,i], label='sEMG ' + str(i+1))

# Agregar etiquetas de los ejes y título del gráfico
ax.set_xlabel('Tiempo (s)', fontsize=12)
ax.set_ylabel('Amplitud', fontsize=12)
# Configurar la cuadrícula con estilo personalizado
ax.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)

# Mostrar la leyenda para identificar cada serie de datos
ax.legend(loc='upper right')

# Mostrar la leyenda de líneas adicionales
ax.legend()

# Agregar una cuadrícula
ax.grid(True)

# Mostrar el gráfico
plt.xlim(min(timeEMG_s1),max(timeEMG_s1))
plt.show()

La siguiente grafica muestra no solo las señales emg sino tambien las repeticiones y las posturas para dar mas contexto:

In [ ]:
# Crear la figura y la cuadrícula de subplots (2 filas, 2 columnas)
fig, axs = plt.subplots(3, 1, figsize=(10, 9))

# Gráfico 1: Seno
for i in range(emg_signals_s1.shape[1]):
    axs[0].plot(timeEMG_s1, emg_signals_s1[:,i])
axs[0].set_title('Señales sEMG', fontsize=14)
axs[0].set_xlabel('Tiempo (s)', fontsize=12)
axs[0].set_ylabel('Amplitud', fontsize=12)
axs[0].grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
axs[0].set_xlim(min(timeEMG_s1),max(timeEMG_s1))


# Gráfico 2: Postura
axs[1].plot(timeEMG_s1, posturas_s1, color='orange')
axs[1].set_title('Posturas', fontsize=14)
axs[1].set_xlabel('Tiempo (s)', fontsize=12)
axs[1].set_ylabel('Postura', fontsize=12)
axs[1].grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
axs[1].set_xlim(min(timeEMG_s1),max(timeEMG_s1))

# Gráfico 3: Repeticion
axs[2].plot(timeEMG_s1, repeticiones_s1, color='green')
axs[2].set_title('Repeticiones', fontsize=14)
axs[2].set_xlabel('Tiempo (s)', fontsize=12)
axs[2].set_ylabel('Repeticion', fontsize=12)
axs[2].grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
axs[2].set_xlim(min(timeEMG_s1),max(timeEMG_s1))

# Ajustar el espacio entre los subplots
plt.tight_layout()

# Mostrar el gráfico con subplots
plt.show()

Es importante notar, que la captura mostrada tiene todos los ejercicios y todas las repeticiones. En la siguiente figura se muestra la grafica resaltando el caso en el cual se lleva solamente los ejercicios asociados a la repetición 2.

In [ ]:
# Numero del estimulo
def indice_postura(estimulo_data, num):
  if isinstance(estimulo_data, pd.DataFrame):
    # Nota: Cuadrar aca...
    return (estimulo_data.index[estimulo_data == num][0],estimulo_data.index[estimulo_data == num][-1])
  else:
    index_cambios = np.where(estimulo_data == num)[0]
    if num == 1:      
      return (0,index_cambios[-1])
    else:
      index_cambios_anterior = np.where(estimulo_data == num - 1)[0]
      return (index_cambios_anterior[-1] + 1,index_cambios[-1])

In [ ]:
# Crear la figura y subgráficos
ind_post2 = indice_postura(posturas_s1,2)
start_index_post2 = ind_post2[0]
end_index_post2 = ind_post2[1]

timeEMG_post2 = timeEMG_s1[start_index_post2:end_index_post2 + 1] 
emgs_s1_post2 = emg_signals_s1[start_index_post2:end_index_post2 + 1,:]
label_s1_post2 = posturas_s1[start_index_post2:end_index_post2 + 1]

# Crear la figura y la cuadrícula de subplots (2 filas, 2 columnas)
fig, axs = plt.subplots(2, 1, figsize=(10, 9))

# Gráfico 1: Seno
for i in range(emgs_s1_post2.shape[1]):
    axs[0].plot(timeEMG_post2, emgs_s1_post2[:,i], label='sEMG ' + str(i+1))
axs[0].set_title('Señales sEMG', fontsize=14)
axs[0].set_xlabel('Tiempo (s)', fontsize=12)
axs[0].set_ylabel('Amplitud', fontsize=12)
axs[0].grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
#plt.xlim(min(timeEMG_post1),max(timeEMG_post1))


# Gráfico 2: Postura
axs[1].plot(timeEMG_post2, label_s1_post2, color='orange')
axs[1].set_title('Posturas', fontsize=14)
axs[1].set_xlabel('Tiempo (s)', fontsize=12)
axs[1].set_ylabel('Postura', fontsize=12)
axs[1].grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
#plt.xlim(min(timeEMG_post1),max(timeEMG_post1))

# Ajustar el espacio entre los subplots
plt.tight_layout()

# Mostrar el gráfico con subplots

plt.show()

## Filtrado

En la figura anterior se pueden apreciar las 10 repeticiones realizadas para la postura 2. Es importante notar que la señal esta muy ruidosa por lo que es necesario realizar un Filtrado.

In [ ]:
# perform 2-order 1Hz low-pass filter
cutoff_freq = 1
nyquist = sample_freq/2
filter_order = 2
filter_type = 'lowpass'
normal_cutoff = cutoff_freq / nyquist
b, a = signal.butter(N = filter_order, 
                     Wn = normal_cutoff, 
                     btype = filter_type)

In [ ]:
emgs_s1_post2_filter = signal.filtfilt(b, a, emgs_s1_post2, axis=0)

In [ ]:
# Crear la figura y subgráficos

# Crear la figura y la cuadrícula de subplots (2 filas, 2 columnas)
fig, axs = plt.subplots(2, 1, figsize=(10, 9))

# Gráfico 1: sEMG
for i in range(emgs_s1_post2_filter.shape[1]):
    axs[0].plot(timeEMG_post2, emgs_s1_post2_filter[:,i], label='sEMG ' + str(i+1))
axs[0].set_title('Señales sEMG', fontsize=14)
axs[0].set_xlabel('Tiempo (s)', fontsize=12)
axs[0].set_ylabel('Amplitud', fontsize=12)
axs[0].grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
# plt.xlim(min(timeEMG_post2),max(timeEMG_post2))


# Gráfico 2: Postura
axs[1].plot(timeEMG_post2, label_s1_post2, color='orange')
axs[1].set_title('Posturas', fontsize=14)
axs[1].set_xlabel('Tiempo (s)', fontsize=12)
axs[1].set_ylabel('Postura', fontsize=12)
axs[1].grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
# plt.xlim(min(timeEMG_post2),max(timeEMG_post2))

# Ajustar el espacio entre los subplots
plt.tight_layout()

# Mostrar el gráfico con subplots

plt.show()

Teniendo en cuenta lo anterior, es necesario aplicarle todo el filtrado a los datos completos. A continuación se muestran las primeras filas del dataframe sin filtrar:

In [ ]:
df.head()

Ahora se procede a filtrar cada una de las columnas de los valores de los sensores sEMG.

In [ ]:
# Aplicando el filtrado en cada columna se tiene
for col in numCols:
    df[col] = signal.filtfilt(b, a, df[col], axis=0)

Los resultados del dataframe filtrado se muestra a continuación:

In [ ]:
df.head()

## Exploración basica del dataset

Inicialmente se explorara el dataset para ver como luce:



In [ ]:
df.info()

In [ ]:
# Sujetos
df['s'].value_counts().sort_index()

Vemos que el numero de muestras por sujeto es aproximadamente el mismo como se puede corroborar en el siguiente grafico de barras

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
sns.countplot(x='s', data=df)
ax.set_title('Muestras por sujeto', fontsize=14)
ax.set_xlabel('Sujeto', fontsize=12)
ax.set_ylabel('Muestras', fontsize=12)
plt.show()

En lo que respecta a la cantidad de repeticiones tenemos:

In [ ]:
df['rep'].value_counts().sort_index()

Hay un total de 10 repeticiones (1 - 10); sin embargo, cuando el sujeto tiene la postura en la posición de descanso, el numero asociado a esta es 0. 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
sns.countplot(x='rep', data=df)
ax.set_title('Repeticiones', fontsize=14)
ax.set_xlabel('Repeticion', fontsize=12)
ax.set_ylabel('Muestras', fontsize=12)
plt.show()

Tal y como se muestra en la grafica anterior, la cantidad de muestras en la posición de descanso es dominante respecto a las muestras asociadas a las repeticiones.

Ahora veamos la cantidad de muestras asociadas a una postura, siendo 0 la postura asociada a la posición de descanso:

In [ ]:
df['label'].value_counts().sort_index()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
sns.countplot(x='label', data=df)
ax.set_title('Posturas', fontsize=14)
ax.set_xlabel('Postura', fontsize=12)
ax.set_ylabel('Muestras', fontsize=12)
plt.show()

Vemos que la mayor cantidad de muestras se da para la pocisión de descanso (0) por lo cual esta es una postura dominante

### Basic information about the dataset and data types

In [ ]:
df.info()

In [ ]:
# Convertir a categorico
df['s'] = pd.Categorical(df['s'])
df['rep'] = pd.Categorical(df['rep'])
df['label'] = pd.Categorical(df['label'])

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
# Completitud
df.isna().sum()

In [ ]:
display(df.nunique())
display(df['s'].unique())
display(df['rep'].unique())
display(df['label'].unique())

### Analisis univariado

In [ ]:
# Display in normal notation instead of scientific
with pd.option_context('float_format', '{:f}'.format):
    display(df.iloc[:,:-2].describe())


In [ ]:
df.columns[1:11]

In [ ]:
# Iterate through each column and plot on a separate subplot
plt.figure(figsize=(15, 10))
i = 1
for column in df.columns[1:3]:
    plt.subplot(3, 2, i)
    sns.histplot(df[column], kde=True)
    plt.title(f'Distribution of {column}')
    plt.xticks(rotation=45)
    i += 1


Para obtener la correlacion vamos a volver el label numerico (de nuevo)

In [ ]:

df['label'] = df['label'].astype('uint8')
# Calculate the correlation matrix
correlation_matrix = df.iloc[:,:-1].corr()

# Print the correlation matrix
print("Correlation Matrix:")
correlation_matrix

In [ ]:
# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

### Distribution

In [ ]:
plt.figure(figsize=(15, 10))

# Iterate through each channel and plot on a separate subplot
for i, column in enumerate(df.columns):
    plt.subplot(3, 2, i+1)
    sns.histplot(df[column], kde=True)
    plt.title(f'Distribution of {column}')
    plt.xticks(rotation=45)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
print(data_base_filter.shape)
print(data_base_filter.isna().sum())

In [ ]:
# Exportamos el dataframe a un archivo CSV
# data_base_filter.to_csv('./datasets/dataset.csv', index=False)

Hasta aqui queda la base de datos estructurada. La siguiente parte consiste en meterla al modelo y rezar para que de.

Aqui no se van a eliminar outlines para evitar la perdida de continuidad de la señal.

## Referencias

* https://github.com/chuawt/eda-starter
* https://www.kaggle.com/code/bextuychiev/my-6-part-powerful-eda-template
* https://community.ibm.com/community/user/ai-datascience/blogs/shivam-solanki1/2020/02/19/eda-exploratory-data-analysis-with-example-in-jupy
* https://github.com/Saba-Gul/Exploratory-Data-Analysis-and-Statistical-Analysis-Notebooks
* https://www.datacamp.com/es/tutorial/pandas-profiling-ydata-profiling-in-python-guide
* https://docs.profiling.ydata.ai/latest/
* https://github.com/Saba-Gul/Exploratory-Data-Analysis-and-Statistical-Analysis-Notebooks/blob/main/Statistics_for_ML.ipynb
* https://github.com/Saba-Gul/Exploratory-Data-Analysis-and-Statistical-Analysis-Notebooks/blob/main/Online_Ed_Adaptability.ipynb
* https://github.com/Saba-Gul/Exploratory-Data-Analysis-and-Statistical-Analysis-Notebooks/blob/main/Heart_Failure_Survival_Classification.ipynb
* https://github.com/akueisara/audio-signal-processing/blob/master/week%204/A4/A4Part2.py